<a href="https://colab.research.google.com/github/jiheeko/Seperation-of-waste-by-materials/blob/main/DeepLabV3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install tensorflow
!pip install opencv-python

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
import cv2
import numpy as np
import os

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
    except RuntimeError as e:
        print(e)

In [ ]:


def load_data(data_path, color=True):
    images = []
    masks = []
    for filename in os.listdir(data_path):
        if filename.startswith("image") and filename.endswith(".jpg"):
            image_path = os.path.join(data_path, filename)

            # Extract mask filename
            mask_filename = f"mask{filename[5:-4]}.jpg"
            mask_path = os.path.join(data_path, mask_filename)

            # Load image
            if color:
                image = cv2.imread(image_path)
            else:
                image = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)

            if image is None:
                print(f"Failed to read image: {image_path}")
                continue

            # Load mask as a grayscale image
            mask = cv2.imread(mask_path, cv2.IMREAD_GRAYSCALE)
            if mask is None:
                print(f"Failed to read mask: {mask_path}")
                continue

            images.append(image)
            masks.append(mask)

    return np.array(images), np.array(masks)

# Load data
train_images, train_masks = load_data('/content/drive/MyDrive/dataset/trainimages')
valid_images, valid_masks = load_data('/content/drive/MyDrive/dataset/validimages')
test_images, test_masks = load_data('/content/drive/MyDrive/dataset/testimages')

# Check the range of mask values
min_mask_value = np.min(train_masks)
max_mask_value = np.max(train_masks)
print(f"Minimum mask value: {min_mask_value}")
print(f"Maximum mask value: {max_mask_value}")

# Clip mask values to the valid range
num_classes = 8
train_masks = np.clip(train_masks - 8, 0, num_classes - 1)
valid_masks = np.clip(valid_masks - 8, 0, num_classes - 1)
test_masks = np.clip(test_masks - 8, 0, num_classes - 1)

# Convert masks to one-hot encoding
train_masks_onehot = tf.keras.utils.to_categorical(train_masks, num_classes=num_classes)
valid_masks_onehot = tf.keras.utils.to_categorical(valid_masks, num_classes=num_classes)
test_masks_onehot = tf.keras.utils.to_categorical(test_masks, num_classes=num_classes)

# Model creation
def create_deeplabv3_model(input_shape=(512, 512, 3), num_classes=8):
    base_model = tf.keras.applications.DenseNet121(input_shape=input_shape, include_top=False)
    x = base_model.output
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Dense(num_classes, activation='softmax')(x)
    model = tf.keras.Model(inputs=base_model.input, outputs=x)
    return model

# Model compilation
model = create_deeplabv3_model()
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Resize images and masks to match the model's input dimensions
input_shape = (512, 512)
train_images_resized = tf.image.resize(train_images, input_shape)

# Resize masks to match the model's input dimensions
train_masks_resized = tf.image.resize(train_masks_onehot, input_shape)

valid_images_resized = tf.image.resize(valid_images, input_shape)
valid_masks_resized = tf.image.resize(valid_masks_onehot, input_shape)
test_images_resized = tf.image.resize(test_images, input_shape)
test_masks_resized = tf.image.resize(test_masks_onehot, input_shape)

# Model training
batch_size = 4
epochs = 20
model.fit(train_images_resized, train_masks_resized, batch_size=batch_size, epochs=epochs, validation_data=(valid_images, valid_masks))






Minimum mask value: 5
Maximum mask value: 231


ResourceExhaustedError: ignored